# Fichiers pour différents tests

## Importation des packages

In [1]:
import warnings
warnings.filterwarnings('ignore')
from pathlib import Path
import requests
from io import BytesIO
from zipfile import ZipFile, BadZipFile
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from pandas_datareader import data
# import pandas_datareader.data as web
from sklearn.datasets import fetch_openml
import seaborn as sns
import sklearn as sk
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale

pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.max_columns', None)


np.random.seed(167)

## Vérification des versions

In [2]:
print('Python  version {}'.format(sys.version))
print('Numpy   version {}'.format(np.__version__))
print('Pandas  version {}'.format(pd.__version__))
print('Seabron version {}'.format(sns.__version__))
print('Sklearn version {}'.format(sk.__version__))

Python  version 3.9.7 (default, Sep 16 2021, 16:59:28) [MSC v.1916 64 bit (AMD64)]
Numpy   version 1.20.3
Pandas  version 1.3.2
Seabron version 0.11.2
Sklearn version 1.0.1


## Tests

### Quandl

In [13]:
import quandl

# Chargement de la clé d'API 
api_key = open("data/nasdaq_api_key.txt", "r")
quandl.ApiConfig.api_key = api_key.readline()

FileNotFoundError: [Errno 2] No such file or directory: 'data/nasdaq_api_key.txt'

In [ ]:
data = quandl.get("WIKI/AAPL", rows=5)
data.head()

,Open,High,Low,Close,Volume,Ex-Dividend,Split Ratio,Adj. Open,Adj. High,Adj. Low,Adj. Close,Adj. Volume
Date,,,,,,,,,,,,
2018-03-21,175.04,175.09,171.26,171.270,35247358.0,0.0,1.0,175.04,175.09,171.26,171.270,35247358.0
2018-03-22,170.00,172.68,168.60,168.845,41051076.0,0.0,1.0,170.00,172.68,168.60,168.845,41051076.0
2018-03-23,168.39,169.92,164.94,164.940,40248954.0,0.0,1.0,168.39,169.92,164.94,164.940,40248954.0
2018-03-26,168.07,173.10,166.44,172.770,36272617.0,0.0,1.0,168.07,173.10,166.44,172.770,36272617.0
2018-03-27,173.68,175.15,166.92,168.340,38962839.0,0.0,1.0,173.68,175.15,166.92,168.340,38962839.0


### pandas_datareader

In [ ]:
from pandas_datareader import data
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
data.get_quote_yahoo(tickers)[['marketCap']]

,marketCap
AAPL,2.460960e+12
MSFT,2.523207e+12
^GSPC,NaN


In [ ]:
# Define the instruments to download. We would like to see Apple, Microsoft and the S&P500 index.
tickers = ['AAPL', 'MSFT', '^GSPC']

# We would like all available data from 01/01/2000 until 12/31/2016.
start_date = '2021-11-15'
end_date = '2016-12-31'

# User pandas_reader.data.DataReader to load the desired data. As simple as that.
panel_data = data.DataReader(tickers, 'yahoo', start_date)

In [ ]:
panel_data['Adj Close'].head()

Symbols,AAPL,MSFT,^GSPC
Date,,,
2021-11-15,150.0,336.070007,4682.799805
2021-11-15,150.0,336.070007,4682.799805


In [ ]:
panel_data['Adj Close'].describe()

Symbols,AAPL,MSFT,^GSPC
count,220.000000,220.000000,220.000000
mean,136.483590,266.799823,4215.275041
std,10.356021,31.873363,259.283378
min,115.819870,210.870728,3700.649902
25%,127.028629,241.307781,3961.669983
50%,134.622185,259.438171,4226.889893
75%,146.336533,293.341766,4424.637451
max,156.461655,336.989990,4701.700195


In [ ]:
all_tickers = data.get_nasdaq_symbols()
all_tickers

,Nasdaq Traded,Security Name,Listing Exchange,Market Category,ETF,Round Lot Size,Test Issue,Financial Status,CQS Symbol,NASDAQ Symbol,NextShares
Symbol,,,,,,,,,,,
A,True,"Agilent Technologies, Inc. Common Stock",N,,False,100.0,False,NaN,A,A,False
AA,True,Alcoa Corporation Common Stock,N,,False,100.0,False,NaN,AA,AA,False
AAA,True,Listed Funds Trust AAF First Priority CLO Bond...,P,,True,100.0,False,NaN,AAA,AAA,False
AAAU,True,Goldman Sachs Physical Gold ETF Shares,P,,True,100.0,False,NaN,AAAU,AAAU,False
AAC,True,Ares Acquisition Corporation Class A Ordinary ...,N,,False,100.0,False,NaN,AAC,AAC,False
...,...,...,...,...,...,...,...,...,...,...,...
ZXZZT,True,NASDAQ TEST STOCK,Q,G,False,100.0,True,N,NaN,ZXZZT,False
ZY,True,Zymergen Inc. - Common Stock,Q,Q,False,100.0,False,N,NaN,ZY,False
ZYME,True,Zymeworks Inc. Common Shares,N,,False,100.0,False,NaN,ZYME,ZYME,False


In [ ]:
all_tickers[~all_tickers['Nasdaq Traded']]

,Nasdaq Traded,Security Name,Listing Exchange,Market Category,ETF,Round Lot Size,Test Issue,Financial Status,CQS Symbol,NASDAQ Symbol,NextShares
Symbol,,,,,,,,,,,
BRK.A,False,Berkshire Hathaway Inc. Common Stock,N,,False,1.0,False,NaN,BRK.A,BRK.A,False


In [ ]:
tickers = all_tickers[(all_tickers['Nasdaq Traded']) & (~all_tickers["Test Issue"])]
tickers

,Nasdaq Traded,Security Name,Listing Exchange,Market Category,ETF,Round Lot Size,Test Issue,Financial Status,CQS Symbol,NASDAQ Symbol,NextShares
Symbol,,,,,,,,,,,
A,True,"Agilent Technologies, Inc. Common Stock",N,,False,100.0,False,NaN,A,A,False
AA,True,Alcoa Corporation Common Stock,N,,False,100.0,False,NaN,AA,AA,False
AAA,True,Listed Funds Trust AAF First Priority CLO Bond...,P,,True,100.0,False,NaN,AAA,AAA,False
AAAU,True,Goldman Sachs Physical Gold ETF Shares,P,,True,100.0,False,NaN,AAAU,AAAU,False
AAC,True,Ares Acquisition Corporation Class A Ordinary ...,N,,False,100.0,False,NaN,AAC,AAC,False
...,...,...,...,...,...,...,...,...,...,...,...
ZWS,True,Zurn Water Solutions Corporation Common Stock,N,,False,100.0,False,NaN,ZWS,ZWS,False
ZY,True,Zymergen Inc. - Common Stock,Q,Q,False,100.0,False,N,NaN,ZY,False
ZYME,True,Zymeworks Inc. Common Shares,N,,False,100.0,False,NaN,ZYME,ZYME,False


In [ ]:
tickers_symbols = tickers["NASDAQ Symbol"].to_list()
tickers_symbols

['A',
 'AA',
 'AAA',
 'AAAU',
 'AAC',
 'AAC=',
 'AAC+',
 'AACG',
 'AACI',
 'AACIU',
 'AACIW',
 'AADI',
 'AADR',
 'AAIC',
 'AAIC-B',
 'AAIC-C',
 'AAIN',
 'AAL',
 'AAMC',
 'AAME',
 'AAN',
 'AAOI',
 'AAON',
 'AAP',
 'AAPL',
 'AAQC',
 'AAQC=',
 'AAQC+',
 'AAT',
 'AATC',
 'AAU',
 'AAWW',
 'AAXJ',
 'AB',
 'ABB',
 'ABBV',
 'ABC',
 'ABCB',
 'ABCL',
 'ABCM',
 'ABEO',
 'ABEQ',
 'ABEV',
 'ABG',
 'ABGI',
 'ABIO',
 'ABM',
 'ABMD',
 'ABNB',
 'ABOS',
 'ABR',
 'ABR-D',
 'ABR-E',
 'ABR-F',
 'ABSI',
 'ABST',
 'ABT',
 'ABTX',
 'ABUS',
 'ABVC',
 'AC',
 'ACA',
 'ACAD',
 'ACAH',
 'ACAHU',
 'ACAHW',
 'ACAQ=',
 'ACB',
 'ACBA',
 'ACBAU',
 'ACBAW',
 'ACBI',
 'ACC',
 'ACCD',
 'ACCO',
 'ACDI=',
 'ACEL',
 'ACER',
 'ACES',
 'ACET',
 'ACEV',
 'ACEVU',
 'ACEVW',
 'ACGL',
 'ACGLN',
 'ACGLO',
 'ACH',
 'ACHC',
 'ACHL',
 'ACHR',
 'ACHR+',
 'ACHV',
 'ACI',
 'ACII',
 'ACII=',
 'ACII+',
 'ACIO',
 'ACIU',
 'ACIW',
 'ACKIT',
 'ACKIU',
 'ACKIW',
 'ACLS',
 'ACM',
 'ACMR',
 'ACN',
 'ACNB',
 'ACOR',
 'ACP',
 'ACP-A',
 'ACQR',
 'A